# 飞桨常规赛：量子电路合成 - 6月第3名方案

### 比赛介绍
本赛题由 “**2020百度之星·程序设计大赛**” 的决赛赛题改编而来，以期为更多开发者提供量子计算领域的学习交流机会。百度自 2017 年起发起了面向全球 AI 技术爱好者的深度学习算法竞赛——百度之星·开发者大赛，大赛的宗旨是为有创新力、专业性强、富有极客精神和团队合作精神的顶级开发者团队提供交流切磋、施展才能的舞台，并为参赛选手提供真实的数据集、深度学习平台飞桨（PaddlePaddle）、完整技术解决方案和一站式 AI 开发平台 AI Studio，降低广大开发者的 AI 学习门槛。2021 年的百度之星·开发者大赛，期待你的加入！

高度发达的 A 星在一次异变中文明即将消失，A 星人将重要信息加密后发送到下一个面临同样异变的欠发达的 C 星，希望能帮助 C 星上的文明躲过这次浩劫。A 星文明高度发达，已经实现了量子计算，故而采用了量子电路来加密信息。C 星接收后深感无力，因为他们的文明只能实现小型的基础量子门，不足以解密该重要信息。此时他们想起了友好星球 B 星上的我们，或许能为他们带来一线生机。而在座的我们能否帮助 C 星文明解决这次危机？

* 科学家分析，神秘信息是经由量子电路加密过的一张图片，我们使用给定的 2 量子比特电路和 3 量子比特电路便可能进行解密。
* 为了不让 C 星坐以待毙，我们要将量子电路分解成 C 星可以实现的基础量子门，从而能帮助 C 星完成解密，完成史诗级的救援任务。
  
比赛详情页：[https://aistudio.baidu.com/aistudio/competition/detail/70](https://aistudio.baidu.com/aistudio/competition/detail/70)

### 赛题说明
比赛包含六道题目，其中前四题为基础题 (满分分别为 1 分，2 分，3分，10 分)，第五题为进阶题 (满分为 25 分)，第六题为挑战题 (满分为 60 分)。选手的比赛最终成绩为六道题目得分总和 (精确到小数点后 4 位) 

* 第一题：单量子比特门近似（1 分）
* 第二题：双量子比特门分解（2 分）
* 第三题：三量子比特门分解（3 分）
* 第四题：三量子比特门无结构分解（10 分）
* 第五题：四量子比特门无结构分解（25 分）
* 第六题：八量子比特门无结构分解（60 分）

比赛使用 **量桨（Paddle Quantum）** 进行搭建量子电路结构，求解参数。

量桨（Paddle Quantum）官网链接：[https://quantum.baidu.com](https://quantum.baidu.com)



### 准备工作

In [1]:
# 环境安装
!pip install paddle-quantum

Looking in indexes: https://mirror.baidu.com/pypi/simple/


In [2]:
# 导入必要的包
import numpy as np
import paddle
import os
from paddle_quantum.circuit import UAnsatz

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


### 解压数据集
数据集解压后的文件夹是中文名，重命名为英文名

In [ ]:
# 解压数据集
!unzip -oq /home/aistudio/data/data71784/飞桨常规赛：量子电路合成.zip -d ./data/
# 重命名数据集文件夹
!mv ./data/飞桨常规赛：量子电路合成 ./data/dataset

In [10]:
import numpy as np
import paddle
from paddle_quantum.circuit import UAnsatz

# 定义量子电路类
class MyCircuit(UAnsatz):

    def gate_fidelity(self, target):
        a = self.U
        b = target
        a = paddle.cast(a, 'float64')
        a.stop_gradient = False
        b = paddle.cast(b, 'float64')
        b.stop_gradient = False
        c = paddle.fluid.layers.matmul(a, b, transpose_y=True)
        trace = paddle.trace(c)
        return paddle.abs(trace) / 2**self.n
    
    @property
    def complexity(self):
        c = 0
        for g, _, _ in self._UAnsatz__history:
            if g == 'u':
                c += 1
            elif g == 'CNOT':
                c += 8
        return c
    
    def output(self, path):
        history = []
        for g, w, p in self._UAnsatz__history:
            if g == 'ry':
                theta = p[0]
                if hasattr(p[0], 'numpy'):
                    theta = p[0].numpy()[0]                    
                history.append('R %d %f\n' % (w[0], np.mod(theta, 2 * np.pi)))
            elif g == 'CNOT':
                history.append('C %d %d\n' % tuple(w))
            else:
                raise OSError('未知的门`%s`' % g)
        with open(path, 'w') as f:
                for i in history:
                    f.write(i)
# 定义网络
class Net(paddle.nn.Layer):
    def __init__(self, n, p, dtype="float64"):
        super(Net, self).__init__()
        self.n = n
        self.theta = self.create_parameter(shape=[p], 
                                           default_initializer=paddle.nn.initializer.Uniform(low=-np.pi/2, high=np.pi/2),
                                           dtype=dtype, is_bias=False)
    def forward(self):
        cir = circuit(self.theta)
        loss = 1 - cir.gate_fidelity(target_tensor)
        return loss, cir

# 定义优化器
class Optimizer:
    def __init__(self, net, learning_rate):
        scheduler = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=LR, T_max=100, eta_min=1e-3, last_epoch=99, verbose=False)
        opt = paddle.optimizer.Adam(learning_rate=scheduler, parameters=net.parameters())
        self.net = net
        self.scheduler = scheduler
        self.opt = opt

    def train(self, max_iters=600, log_iter=50, threshold=1e-8, display=True):
        last = 1
        for itr in range(1, max_iters + 1):
            loss, cir = self.net()
            loss.backward()
            self.opt.minimize(loss)
            self.opt.clear_grad()
            self.scheduler.step()
            if itr % log_iter == 0:
                l = loss.numpy()[0]
                if display:
                    print("iter:", "%3d" % itr, "  loss:", "%.4f" % l, "%.2e" % (l - last))
                if abs(l - last) < threshold:
                    break
                last = l
        return last

# 定义可变量子电路类
class VariableCircuit(MyCircuit):
    def __init__(self, n, theta, controls=[]):
        super().__init__(n)
        self.controls = controls
        self.init(theta)

    def init(self, theta):
        offset = 0
        for i in range(self.n):
            self.ry(theta[offset+i], i)
        offset += self.n

        for a, b in self.controls:
            self.cnot([a, b])
            self.ry(theta[offset+0], a)
            self.ry(theta[offset+1], b)
            offset += 2
        self.params_size = offset

![](https://ai-studio-static-online.cdn.bcebos.com/89dd7e77cedf401983dcb43d931892743c2d8756a86c485985924457e9dcd7b4)

```
import numpy as np
import paddle
from paddle_quantum.circuit import UAnsatz

# 设置角度参数 theta = pi
theta = np.array([np.pi])
    
# 我们需要将 Numpy array 转换成 Paddle 中的 Tensor
theta = paddle.to_tensor(theta)

# 设置计算所需的量子比特数量
num_qubits = 1

# 初始化我们的单比特量子电路
cir = UAnsatz(num_qubits)

# 在第一个量子比特(第0号量子比特)的位置上施加一个 Rx 旋转门， 角度为 pi
which_qubit = 0
cir.rx(theta, which_qubit)

# 打印出这个量子门
# 转换成 numpy 
print('量子门的矩阵表达式为：')
print(cir.U.numpy())
```


![](https://ai-studio-static-online.cdn.bcebos.com/3eadc1397b554440a9d2601eacdebbc244317a12c14d47bca5135e308dbeba4a)


```
import numpy as np
import paddle
from paddle_quantum.circuit import UAnsatz

# 设置角度参数 theta 
theta = np.full([4], np.pi)
    
# 我们需要将 Numpy array 转换成 Paddle 中的 Tensor
theta = paddle.to_tensor(theta)

# 初始化量子电路
num_qubits = 2
cir = UAnsatz(num_qubits)

# 添加单比特旋转门
cir.ry(theta[0], 0)
cir.ry(theta[1], 1)

# 添加两比特门
cir.cnot([0, 1])

# 添加单比特旋转门
cir.ry(theta[2], 0)
cir.ry(theta[3], 1)

print('图中量子神经网络 U(theta=pi) 的矩阵表达式是:')
print(cir.U.numpy().real)
```

![](https://ai-studio-static-online.cdn.bcebos.com/6e9f5d5d98a34f298e1961481163820c7001a85e8823418e94ed73d59c057c15)

根据量浆官网中的[快速入门](https://qml.baidu.com/quick-start/quantum-computing-fundamentals.html#3-%E5%8D%95%E9%87%8F%E5%AD%90%E6%AF%94%E7%89%B9%E9%97%A8)，熟悉单量子比特门相关内容进行求解

In [11]:
n = 1 #单量子比特电路
target = 1 / 2**0.5 * np.array([[1, -1], [1, 1]]) #目标酉矩阵
target_tensor = paddle.to_tensor(target) #转化为tensor
LR = 0.1 #学习率
ITR = 1000 #最大迭代次数

#定义电路结构
def circuit(theta):
    cir = MyCircuit(n)
    cir.ry(theta[0], 0) #单个旋转门
    return cir

In [12]:
net = Net(n, 1) #定义网络
opt = Optimizer(net, LR) #定义自动优化器
opt.train(ITR, threshold=1e-15) #优化参数

theta_opt = net.theta.numpy() #优化好的参数
print("优化后的参数 theta:\n", theta_opt)

cir = circuit(net.theta) #将优化好的参数加载到量子电路中
cir.output('Question_1_Answer.txt') #输出结果到文件

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/framework.py:706: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


iter:  50   loss: 0.0065 -9.93e-01
iter: 100   loss: 0.0001 -6.38e-03
iter: 150   loss: 0.0000 -1.34e-04
iter: 200   loss: 0.0000 -6.24e-07
iter: 250   loss: 0.0000 -4.48e-10
iter: 300   loss: 0.0000 -2.12e-11
iter: 350   loss: 0.0000 -5.60e-14
iter: 400   loss: 0.0000 -8.88e-16
优化后的参数 theta:
 [1.57079632]


![](https://ai-studio-static-online.cdn.bcebos.com/d952c29d2ecd440bb9ff5bc030cd03ffcf4d52ca08004496ba1be5897b5f61cb)


In [ ]:
n = 2
target = np.loadtxt('/home/aistudio/data/dataset/Question_2_Unitary.txt') #读取目标实数酉矩阵U
target_tensor = paddle.to_tensor(target)
LR = 0.05
ITR = 1000

def circuit(theta):
    cir = MyCircuit(n)
    cir.ry(theta[0], 0)
    cir.ry(theta[1], 1)
    cir.cnot([0, 1])
    cir.ry(theta[2], 0)
    cir.ry(theta[3], 1)
    return cir

In [ ]:
net = Net(n, 4)
opt = Optimizer(net, LR)
opt.train(ITR, threshold=1e-15)

theta_opt = net.theta.numpy()
print("优化后的参数 theta:\n", theta_opt)

cir = circuit(net.theta)
cir.output('Question_2_Answer.txt')

iter:  50   loss: 0.4692 -5.31e-01
iter: 100   loss: 0.0034 -4.66e-01
iter: 150   loss: 0.0000 -3.34e-03
iter: 200   loss: 0.0000 -1.82e-05
iter: 250   loss: 0.0000 -2.35e-06
iter: 300   loss: 0.0000 -8.36e-08
iter: 350   loss: 0.0000 -1.36e-09
iter: 400   loss: 0.0000 -2.64e-12
iter: 450   loss: 0.0000 -1.92e-13
iter: 500   loss: 0.0000 -3.33e-16
优化后的参数 theta:
 [-0.43582512  0.79511961 -2.79500593  0.42428991]


![](https://ai-studio-static-online.cdn.bcebos.com/ec50e9da79dc4cf29419be08acd1e2aa1545175836c54d84bd57a3c24af29f89)


In [ ]:
n = 3
target = np.loadtxt('/home/aistudio/data/dataset/Question_3_Unitary.txt')
target_tensor = paddle.to_tensor(target)
LR = 0.02
ITR = 1000

def circuit(theta):
    # 初始化 n 个量子比特的量子电路
    cir = MyCircuit(n)
    cir.ry(theta[0], 0)
    cir.ry(theta[1], 1)
    cir.ry(theta[2], 2)
    cir.cnot([0, 1])
    cir.cnot([1, 2])
    cir.ry(theta[3], 0)
    cir.ry(theta[4], 2)
    return cir

In [ ]:
net = Net(n, 5)
opt = Optimizer(net, LR)
opt.train(ITR, threshold=1e-15)

theta_opt = net.theta.numpy()
print("优化后的参数 theta:\n", theta_opt)

cir = circuit(net.theta)
cir.output('Question_3_Answer.txt')

iter:  50   loss: 0.7453 -2.55e-01
iter: 100   loss: 0.2235 -5.22e-01
iter: 150   loss: 0.0351 -1.88e-01
iter: 200   loss: 0.0204 -1.48e-02
iter: 250   loss: 0.0120 -8.40e-03
iter: 300   loss: 0.0007 -1.12e-02
iter: 350   loss: 0.0000 -7.26e-04
iter: 400   loss: 0.0000 -9.48e-06
iter: 450   loss: 0.0000 -3.29e-06
iter: 500   loss: 0.0000 -2.76e-06
iter: 550   loss: 0.0000 -2.54e-08
iter: 600   loss: 0.0000 -6.69e-11
iter: 650   loss: 0.0000 -2.95e-11
iter: 700   loss: 0.0000 -1.66e-11
iter: 750   loss: 0.0000 1.33e-15
iter: 800   loss: 0.0000 -9.99e-16
优化后的参数 theta:
 [-2.30239562 -1.78154508 -0.33809543 -1.46190544  1.98867992]


![](https://ai-studio-static-online.cdn.bcebos.com/412a7f084c3b439782b2ef8c5d9b6f984b02b9b8c4ee4296b39941a175fa62ee)


In [ ]:
n = 3
target = np.loadtxt('/home/aistudio/data/dataset/Question_4_Unitary.txt')
target_tensor = paddle.to_tensor(target)
LR = 0.1
ITR = 5000

In [ ]:
def circuit(theta):
    # conts = [(1,2), (1,2), (0,1), (0,2), (0,1)]
    conts = [(1,2), (1,2),(1,2),(0,2), (0,1)]
    cir = VariableCircuit(n, theta, conts)
    return cir

net = Net(n, 15)
cir = circuit(net.theta)
print(cir.complexity)
opt = Optimizer(net, LR)
opt.train(ITR, threshold=1e-20)

40
iter:  50   loss: 0.4544 -5.46e-01
iter: 100   loss: 0.3331 -1.21e-01
iter: 150   loss: 0.3317 -1.42e-03
iter: 200   loss: 0.3317 -6.83e-06


In [ ]:
theta_opt = net.theta.numpy()
theta_opt = np.mod(theta_opt, 2 * np.pi)
print("优化后的参数 theta:\n", theta_opt / np.pi)

cir = circuit(net.theta)
cir.output('Question_4_Answer.txt')

优化后的参数 theta:
 [0.34226911 1.49999981 0.4999999  0.77098571 0.22901438 1.92208134
 1.39719538 0.34248397 1.50000001 1.50000006 0.69368954 1.21107806
 1.7403326  1.97489432 0.26339706]


![](https://ai-studio-static-online.cdn.bcebos.com/93767e288e02417393405d6376e63a485c89871270374cf2af8deb0121366ce1)


In [ ]:
n = 4
target = np.loadtxt('/home/aistudio/data/dataset/Question_5_Unitary.txt')
target_tensor = paddle.to_tensor(target)
LR = 0.1
ITR = 1000

def circuit(theta):
    conts = [(1, 2)] #开头(1, 2)和结尾(3, 0)为尝试过程中最优结构的开头和结尾
    for i in range(7):
        for j in range(i%2, n, 2):
            conts.append((j%n, (j+1)%n))
    conts.append((3, 0))
    cir = VariableCircuit(n, theta, conts)
    return cir

In [ ]:
while 1: #循环运行直至满足及格条件
    net = Net(n, 36)
    opt = Optimizer(net, LR)
    l = opt.train(ITR)
    if l < 0.3:
        break

iter:  50   loss: 0.5351 -4.65e-01
iter: 100   loss: 0.4121 -1.23e-01
iter: 150   loss: 0.4038 -8.28e-03
iter: 200   loss: 0.4016 -2.17e-03
iter: 250   loss: 0.3978 -3.86e-03
iter: 300   loss: 0.3492 -4.86e-02
iter: 350   loss: 0.3282 -2.10e-02


In [ ]:
theta_opt = net.theta.numpy()
print("优化后的参数 theta:\n", theta_opt)

cir = circuit(net.theta)
cir.output('Question_5_Answer.txt')

In [ ]:
theta_opt = net.theta.numpy()
print("优化后的参数 theta:\n", theta_opt)

cir = circuit(net.theta)
cir.output('Question_6_Answer.txt')

### 压缩文件

In [ ]:
!zip Answer.zip Question_?_Answer.txt

updating: Question_1_Answer.txt (stored 0%)
updating: Question_2_Answer.txt (deflated 12%)
updating: Question_3_Answer.txt (deflated 21%)
updating: Question_4_Answer.txt (deflated 49%)
updating: Question_5_Answer.txt (deflated 58%)
updating: Question_6_Answer.txt (deflated 58%)


### 参考项目

量浆官网快速入门：[https://qml.baidu.com/quick-start/overview.html](https://qml.baidu.com/quick-start/quantum-computing-fundamentals.html#3-%E5%8D%95%E9%87%8F%E5%AD%90%E6%AF%94%E7%89%B9%E9%97%A8)

量浆官网教程：[https://qml.baidu.com/tutorials/overview.html](https://qml.baidu.com/tutorials/overview.html)

飞桨常规赛：量子电路合成 5月第1名方案（作者：Mr.郑先生_）：[https://aistudio.baidu.com/aistudio/projectdetail/2025686?channel=0&channelType=0&shared=1](https://aistudio.baidu.com/aistudio/projectdetail/2025686?channel=0&channelType=0&shared=1)

飞桨常规赛：量子电路合成 4月第1名方案（作者：bnpzsx）：[https://aistudio.baidu.com/aistudio/projectdetail/1932455?channel=0&channelType=0&shared=1](https://aistudio.baidu.com/aistudio/projectdetail/1932455?channel=0&channelType=0&shared=1)